# Develop Model Driver

In this notebook, we will develop the API that will call our model. This module initializes the model, transforms the input so that it is in the appropriate format and defines the scoring method that will produce the predictions. The API will expect the input to be in JSON format. Once a request is received, the API will use the request body to score the question text. There are two main functions in the API. The first function loads the model and returns a scoring function. The second function process the question text and uses the first function to score it.

In [1]:
import warnings
warnings.filterwarnings(action="ignore", message="numpy.dtype size changed")

import pandas as pd
import logging
import json
from utilities import text_to_json

We use the writefile magic to write the contents of the below cell to driver.py which includes the driver methods.

In [2]:
%%writefile driver.py

import lightgbm as lgb
import timeit as t
import logging
from duplicate_model import DuplicateModel

model_path = 'model.pkl'
questions_path = 'questions.tsv'
logger = logging.getLogger("model_driver")

def create_scoring_func():
    """ Initialize Model Object 
    """   
    start = t.default_timer()
    DM = DuplicateModel(model_path, questions_path)
    end = t.default_timer()
    
    loadTimeMsg = "Model object loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)
    
    def call_model(text):
        preds = DM.score(text)  
        return preds
    
    return call_model

def get_model_api():
    logger = logging.getLogger("model_driver")
    scoring_func = create_scoring_func()
    
    def process_and_score(inputString):
        """ Classify the input using the loaded model
        """
        start = t.default_timer()
        responses = scoring_func(inputString)
        end = t.default_timer()
        
        logger.info("Predictions: {0}".format(responses))
        logger.info("Predictions took {0} ms".format(round((end-start)*1000, 2)))
        return (responses, "Computed in {0} ms".format(round((end-start)*1000, 2)))
    return process_and_score

def version():
    return lgb.__version__

Writing driver.py


Let's test the module.

In [3]:
logging.basicConfig(level=logging.DEBUG)

We run the driver.py which will bring the imports and functions into the context of the notebook.

In [4]:
%run driver.py

Now, let's use one of the duplicate questions to test our driver.

In [5]:
dupes_test_path = 'dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
text_to_score = dupes_test.iloc[0,4]
text_to_score

"get total number of items on json object?.  possible duplicate: length of javascript object (ie. associative array)   i have an object similar to this one:  i'm trying to get it's length, the problem is that jsonarray.length returns 5 instead of 3 (which is the total items it has). the array is relatively long (has 1000x2000 items) and this must be done a lot of times every second. how can i get the number of items more efficiently?"

Here, we convert our text for the format that will be required by the Flask application that will use the functions in the driver.

In [6]:
jsontext = text_to_json(text_to_score)
json_load_text = json.loads(jsontext)
body = json_load_text['input']

In [7]:
predict_for = get_model_api()

INFO:model_driver:Model object loading time: 378.67 ms


In [8]:
resp = predict_for(body)

INFO:model_driver:Predictions: [(11922383, 11922384, 0.5303780927518741), (5223, 6700, 0.3147833277520598), (126100, 4889658, 0.17501291628511934), (1069666, 1069840, 0.09867350149915492), (4935632, 4935684, 0.03558077142398663), (12953704, 12953750, 0.008518479006562647), (19590865, 19590901, 0.002929205659738695), (6491463, 6491621, 0.0027232369677284976), (14220321, 14220323, 0.0019885224026778992), (122102, 122704, 0.001893584053682761), (85992, 86014, 0.0015529544139815355), (23667086, 23667087, 0.0012226445727317346), (1885557, 1885660, 0.0010271727668257863), (171251, 171256, 0.0009754234115706664), (901115, 901144, 0.0009454486631845099), (684672, 684692, 0.0008335616599598791), (4057440, 4060176, 0.0007165840563749822), (1068834, 1144249, 0.0006732119754763038), (728360, 728694, 0.000597171407530765), (5627284, 5627301, 0.0005865098737299449), (111102, 111111, 0.0005369210664999444), (4255472, 4255480, 0.0005261610115592135), (4616202, 4616273, 0.0004944634404607464), (1129216

Next, we move on to [building our docker image](03_Build_Image.ipynb).